In [12]:
import argparse, os, time
import urllib3, random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re

# Open my page

In [13]:
def open_my_page():
    login_user_cell_id = 'username'
    login_pw_cell_id = 'password'
    # launch browser
    browser = webdriver.Chrome()
    # open start page
    browser.get("https://www.linkedin.com/login")
    time.sleep(3)
    #
    my_user = 'ing.bianco.roberto@gmail.com'
    emailElement = browser.find_element_by_id( login_user_cell_id )
    emailElement.send_keys(my_user)
    time.sleep(.5)
    #
    my_pw = 'password01'
    pwElement = browser.find_element_by_id( login_pw_cell_id )
    pwElement.send_keys(my_pw)
    time.sleep(.5)
    # Submit form
    pwElement.submit()
    #
    return browser

# Aux

In [14]:
def extract_tag_from_html(page, tag_string, parameter, parameter_value):
    # returns the tag where <tag_string parameter = 'parameter_value'>
    result = []
    tags = page.find_all(tag_string)
    for tag in tags:
        tag_parameter = tag.get(parameter)
        if tag_parameter:
            if parameter_value in tag_parameter:
                result.append(tag)
    return result

In [15]:
def between_tag_markers(page, tag_string, parameter, parameter_value):
    
    tag_list = extract_tag_from_html(page, tag_string, parameter, parameter_value)
    result = []
    for tag in tag_list:
        # transforms into a unique string and removes \n
        tag_string = re.sub('\n', '', str(tag))
        between_tag = re.findall('>(.*)<', tag_string)
        result.append( between_tag[0].strip() )
    return result

In [16]:
def within_tag_marker(page, tag_string, parameter, parameter_value, field_of_interest):
    # reports the field of interest within the tag
    tag_list = extract_tag_from_html(page, tag_string, parameter, parameter_value)
    result = []
    for tag in tag_list:
        out = tag.get(field_of_interest)
        result.append(out)
    return result

In [17]:
def scroll_to_bottom(browser):
    # scrolls to bottom to load/show entire page
    
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
            lastCount = lenOfPage
            time.sleep(1.5)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True

# Collect links from my connections

In [18]:
def get_my_connection_links(browser):
    #move to connections
    browser.get("https://www.linkedin.com/mynetwork/invite-connect/connections")
    time.sleep(3)
    # scroll to bottom
    scroll_to_bottom(browser)
    # Page
    page = BeautifulSoup(browser.page_source)
    # scan for links
    links = within_tag_marker(page, 'a','class','mn-connection-card__link','href')
    #
    return links

# Collect data from first order link

In [19]:
class Connection:
   
    def __init__(self, name, link, occupation, location, number_mutual_connections, \
                 date_connected,see_connections_link):
        self.name = name
        self.link = link
        self.occupation = occupation
        self.location = location
        self.number_mutual_connections = number_mutual_connections
        self.date_connected = date_connected
        self.see_connections_link = see_connections_link

In [67]:
def connection_to_row(conn):
    return [conn.name, conn.link, conn.occupation, conn.location, conn.number_mutual_connections,\
        conn.date_connected, conn.see_connections_link]

In [61]:
# AUX
def get_date_connected(browser, link):
    root_link = 'https://linkedin.com'
    browser.get(root_link + link + 'detail/contact-info/')
    time.sleep(1.5)
    page = BeautifulSoup(browser.page_source)
    #
    #date_connected = between_tag_markers(page,'span', 'class','pv-contact-info__contact-item')[0]
    section = extract_tag_from_html(page, 'section', 'class', \
            'ci-connected' )
    #
    section_string = str(section[0])
    section_string = re.sub('\n', '', section_string)
    between_tag = re.findall('<span class(:?.*?)>(.*?)<', section_string)
    date_connected = between_tag[0][1].strip()
    #
    return date_connected

In [21]:
def my_list_proc(input_list, null_value):
    if input_list == []:
        return null_value
    if len(input_list) == 1:
        return input_list[0]
    else:
        return input_list

In [28]:
def get_first_order_connection(browser, link):
    root_link = 'https://linkedin.com'
    browser.get(root_link + link)
    time.sleep(random.uniform(2,3))
    page = BeautifulSoup(browser.page_source)
    # scan page
    name = between_tag_markers(page,'h1','class','pv-top-card-section__name')[0]
    occupation = between_tag_markers(page, 'h2','class','pv-top-card-section__headline')[0]
    location = between_tag_markers(page,'h3','class','pv-top-card-section__location')[0]
    number_mutual_connections = \
        my_list_proc(between_tag_markers(page,'h3','class','pv-highlight-entity__primary-text'),\
                    '0 Mutual connections')
    see_connections_link = my_list_proc(\
            within_tag_marker(page,'a','data-control-name','topcard_view_all_connections','href'),\
                    '')
        
    # extract date_connected
    date_connected = get_date_connected(browser, link)
    # Connection
    c = Connection(name, link, occupation, location, number_mutual_connections, \
                   date_connected, see_connections_link)
    #
    return c

# Main script

In [23]:
browser = open_my_page()

In [24]:
#
#time.sleep(5)
#
links = get_my_connection_links(browser)

In [25]:
print('Number of links:' + str(len(links)))

Numer of links:293


In [82]:
first_connections = []

for k in range(len(links)):
    print( 'Contact ' + str(k+1) + ' of ' + str(len(links)) )
    #
    time.sleep(random.uniform(0.5, 1.2))
    #
    conn = get_first_order_connection(browser, links[k])
    #
    first_connections.append( conn )
 

Contact 1 of 293
Contact 2 of 293
Contact 3 of 293
Contact 4 of 293
Contact 5 of 293
Contact 6 of 293
Contact 7 of 293
Contact 8 of 293
Contact 9 of 293
Contact 10 of 293
Contact 11 of 293
Contact 12 of 293
Contact 13 of 293
Contact 14 of 293
Contact 15 of 293
Contact 16 of 293
Contact 17 of 293
Contact 18 of 293
Contact 19 of 293
Contact 20 of 293
Contact 21 of 293
Contact 22 of 293
Contact 23 of 293
Contact 24 of 293
Contact 25 of 293
Contact 26 of 293
Contact 27 of 293
Contact 28 of 293
Contact 29 of 293
Contact 30 of 293
Contact 31 of 293
Contact 32 of 293
Contact 33 of 293
Contact 34 of 293
Contact 35 of 293
Contact 36 of 293
Contact 37 of 293
Contact 38 of 293
Contact 39 of 293
Contact 40 of 293
Contact 41 of 293
Contact 42 of 293
Contact 43 of 293
Contact 44 of 293
Contact 45 of 293
Contact 46 of 293
Contact 47 of 293
Contact 48 of 293
Contact 49 of 293
Contact 50 of 293
Contact 51 of 293
Contact 52 of 293
Contact 53 of 293
Contact 54 of 293
Contact 55 of 293
Contact 56 of 293
C

# Write CSV

In [110]:
import csv
with open('first_connections.csv', 'w') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=';')
    errs = []
    for k in range(len(first_connections)):
        try:
            contact = first_connections[k]
            csv_writer.writerow( connection_to_row(contact) )
        except:
            errs.append(k)
print(errs)

[]


In [103]:
for kerr in errs:
    print(connection_to_row(first_connections[kerr]) )

['James Edward Tweed  ☁', '/in/jetweed/', 'Network Specialist at Motorola Solutions PRINCE2 Practitioner | ITIL Certified', 'Hatfield, Hertfordshire, United Kingdom', '2 Mutual Connections', 'June 15, 2016', '/search/results/people/?facetConnectionOf=%5B%22ACoAABEXuhIBNmzs9__hRb5Bq5SSasmxSdiekgk%22%5D&facetNetwork=%5B%22F%22%2C%22S%22%5D&origin=MEMBER_PROFILE_CANNED_SEARCH']
['Grace Zeng(曾宇)', '/in/headhunter9925/', 'Management Consultant', 'Hong Kong', '17 Mutual Connections', 'April 24, 2016', '']
['Alessio Lombardi ❇', '/in/alessio-lombardi/', 'Senior  Manager  - South Europe Sales at VI-grade', 'Turin Area, Italy', '46 Mutual Connections', 'August 5, 2013', '']


In [105]:
first_connections[errs[0]].name = 'James Edward Tweed'

In [107]:
first_connections[errs[0]].name = 'James Edward Tweed'

In [109]:
first_connections[errs[2]].name = 'Alessio Lombardi'